In [58]:
import pandas as pd
from sklearn.metrics import f1_score
from tqdm import tqdm
import torch
import numpy as np

In [73]:
from transformers import BertTokenizer, AutoModelForSequenceClassification
from torch.optim import  AdamW

In [60]:
from openprompt.prompts import ManualTemplate
from openprompt.prompts import ManualVerbalizer
from openprompt.data_utils import InputExample
from openprompt.plms import load_plm
from openprompt import PromptForClassification, PromptDataLoader

In [61]:
plm_object = load_plm("bert", "microsoft/MiniLM-L12-H384-uncased")
tokenizer = plm_object[1]

Some weights of BertForMaskedLM were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [68]:
classes = [
    "toxic",
    "nontoxic"
]

In [74]:
prompt_template_1 = ManualTemplate(
    text = '{"placeholder":"text_a"}. Were there swear words in the previous sentence? {"mask"}',
    tokenizer = tokenizer,
)

prompt_verbalizer_1 = ManualVerbalizer(
    classes = classes,
    label_words = {
        "toxic": ["yes"],
        "nontoxic": ["no"],
    },
    tokenizer = tokenizer,
)

prompt_template_2 = ManualTemplate(
    text = '{"placeholder":"text_a"}. On a scale of 1-5, how rude was that sentence? {"mask"}',
    tokenizer = tokenizer,
)

prompt_verbalizer_2 = ManualVerbalizer(
    classes = classes,
    label_words = {
        "toxic": ["4", "5"],
        "nontoxic": ["1", "2", "3"],
    },
    tokenizer = tokenizer,
)

prompt_template_3 = ManualTemplate(
    text = '{"placeholder":"text_a"}. What was the author of the previous sentence feeling? {"mask"}',
    tokenizer = tokenizer,
)

prompt_verbalizer_3 = ManualVerbalizer(
    classes = classes,
    label_words = {
        "toxic": ["anger", "frustration", "sad", "depressed"],
        "nontoxic": ["content", "happy", "calm", "nothing"],
    },
    tokenizer = tokenizer,
)

prompt_template_4 = ManualTemplate(
    text = '{"placeholder":"text_a"}. Would you kiss your mother with that mouth? {"mask"}',
    tokenizer = tokenizer,
)

prompt_verbalizer_4 = ManualVerbalizer(
    classes = classes,
    label_words = {
        "toxic": ["no"],
        "nontoxic": ["yes"],
    },
    tokenizer = tokenizer,
)

prompt_template_5 = ManualTemplate(
    text = '{"placeholder":"text_a"}. If that sentence were spoken out, how loud would it be on a scale of 1-5? {"mask"}',
    tokenizer = tokenizer,
)


prompt_verbalizer_5 = ManualVerbalizer(
    classes = classes,
    label_words = {
        "toxic": ["4", "5"],
        "nontoxic": ["1", "2", "3"],
    },
    tokenizer = tokenizer,
)

prompt_template_6 = ManualTemplate(
    text = 'Alice: How are you? Bob: {"placeholder":"text_a"}. Alice: That\'s {"mask"}',
    tokenizer = tokenizer,
)


prompt_verbalizer_6 = ManualVerbalizer(
    classes = classes,
    label_words = {
        "toxic": ["rude", "terrible", "bad"],
        "nontoxic": ["nice", "good", "alright"],
    },
    tokenizer = tokenizer,
)

prompt_template_7 = ManualTemplate(
    text = 'If someone said {"placeholder":"text_a"} that would be {"mask"}',
    tokenizer = tokenizer,
)

prompt_verbalizer_7 = ManualVerbalizer(
    classes = classes,
    label_words = {
        "toxic": ["rude", "terrible", "bad"],
        "nontoxic": ["nice", "good", "alright"],
    },
    tokenizer = tokenizer,
)

prompt_template_8 = ManualTemplate(
    text = 'If I agree with {"placeholder":"text_a"} then I am {"mask"}',
    tokenizer = tokenizer,
)

prompt_verbalizer_8 = ManualVerbalizer(
    classes = classes,
    label_words = {
        "toxic": ["rude", "terrible", "dick", "horrible"],
        "nontoxic": ["nice", "good", "alright", "calm", "composed"],
    },
    tokenizer = tokenizer,
)

prompt_template_9 = ManualTemplate(
    text = '''
    YOU GAY ASSHOLE MOTHERFUCKER, THAT'S BULLSHIT. Is this toxic? yes
    Well, that's alright, you didn't know. Is this toxic? no
    {"placeholder":"text_a"}. Is this toxic? {"mask"}
    ''',
    tokenizer = tokenizer,
)

prompt_verbalizer_9 = ManualVerbalizer(
    classes = classes,
    label_words = {
        "toxic": ["yes"],
        "nontoxic": ["no"],
    },
    tokenizer = tokenizer,
)

prompt_template_10 = ManualTemplate(
    text = 'As a Reddit mod would you ban this comment: {"placeholder":"text_a"}? {"mask"}',
    tokenizer = tokenizer,
)

prompt_verbalizer_10 = ManualVerbalizer(
    classes = classes,
    label_words = {
        "toxic": ["yes"],
        "nontoxic": ["no"],
    },
    tokenizer = tokenizer,
)

In [75]:
class PromptClassifier:
    def __init__(self, template, verbalizer, plm_obj):
        self.plm, self.tokenizer, self.model_config, self.WrapperClass = plm_obj
        self.template = template
        self.verbalizer = verbalizer
        self.model = PromptForClassification(
            template = template,
            plm = self.plm,
            verbalizer = verbalizer,
        )
        self.criterion = torch.nn.BCELoss()
        self.optim = AdamW(self.model.parameters(), lr=1e-2)
    
    def create_data_loader(self, dataset):
        return PromptDataLoader(
            dataset = dataset,
            tokenizer = self.tokenizer,
            template = self.template,
            tokenizer_wrapper_class = self.WrapperClass,
        )

    def train(self, dataset):
        for batch in tqdm(self.create_data_loader(dataset)):
            logits = self.model(batch)
            preds = torch.argmax(logits, dim = -1)
            loss = self.criterion(preds.float(), batch["label"].float())
            loss = torch.autograd.Variable(loss, requires_grad = True)
            self.optim.step()
            self.optim.zero_grad()
            loss.backward()

    def evaluate(self, dev_dataset):
        self.model.eval()
        y_hats = []
        ys = []
        with torch.no_grad():
            for batch in tqdm(self.create_data_loader(dev_dataset)):
                logits = self.model(batch)
                preds = torch.argmax(logits, dim = -1)
                y_hats.append(preds.item())
                ys.append(batch["label"].item())    
        return f1_score(ys, y_hats)

In [76]:
df = pd.read_csv("jigsaw-train.csv")
dev_df = pd.read_csv("jigsaw-dev.csv")

In [77]:
best_classifiers = []
for i in tqdm(range(10)):
    dataset_0 = [
        InputExample(
            guid=idx,
            text_a=row["comment_text"],
            label=row["toxic"]
        ) for idx, row in df[df["toxic"] == 0].sample(5).iterrows()
    ]

    dataset_1 = [
        InputExample(
            guid=idx,
            text_a=row["comment_text"],
            label=row["toxic"]
        ) for idx, row in df[df["toxic"] == 1].sample(5).iterrows()
    ]

    dataset = dataset_0 + dataset_1

    dev_dataset_0 = [
        InputExample(
            guid=idx,
            text_a=row["comment_text"],
            label=row["toxic"]
        ) for idx, row in dev_df[dev_df["toxic"] == 0].sample(5).iterrows()
    ]

    dev_dataset_1 = [
        InputExample(
            guid=idx,
            text_a=row["comment_text"],
            label=row["toxic"]
        ) for idx, row in dev_df[dev_df["toxic"] == 1].sample(5).iterrows()
    ]

    dev_dataset = dev_dataset_0 + dev_dataset_1
    
    classifiers = []
    f1_scores = []
    for i in range(1, 11):
        prompt_classifier = PromptClassifier(eval("prompt_template_" + str(i)), eval("prompt_verbalizer_" + str(i)), plm_object)
        prompt_classifier.train(dataset)
        classifiers.append(prompt_classifier)
        score = prompt_classifier.evaluate(dev_dataset)
        f1_scores.append(score)
    best_classifiers.append(np.argmax(f1_scores))
best_model_index = max(set(best_classifiers), key=best_classifiers.count)
best_model = classifiers[best_model_index]
best_model_index

  0%|                                                            | 0/10 [00:00<?, ?it/s]
tokenizing: 10it [00:00, 302.87it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.95it/s]

tokenizing: 10it [00:00, 675.36it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.25it/s]

tokenizing: 10it [00:00, 297.58it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.98it/s]

tokenizing: 10it [00:00, 602.08it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.69it/s]

tokenizing: 10it [00:00, 275.56it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.50it/s]

tokenizing: 10it [00:00, 441.64it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.89it/s]

tokenizing: 10it [00:00, 260.30it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.57it/s]

tokenizing: 10i

100%|███████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.70it/s]

tokenizing: 10it [00:00, 197.04it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.55it/s]

tokenizing: 10it [00:00, 412.08it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.58it/s]

tokenizing: 10it [00:00, 212.60it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:07<00:00,  1.40it/s]

tokenizing: 10it [00:00, 457.39it/s]

 10%|█████                                              | 1/10 [01:58<17:43, 118.19s/it]
tokenizing: 10it [00:00, 476.87it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.52it/s]

tokenizing: 10it [00:00, 386.27it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.61it/s]

tokenizing: 10it [00:00, 412.14it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:08<00:00,  1.15it/s]

tokenizing: 10i

100%|███████████████████████████████████████████████████| 10/10 [00:08<00:00,  1.15it/s]

tokenizing: 10it [00:00, 375.01it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:07<00:00,  1.42it/s]

tokenizing: 10it [00:00, 335.66it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:08<00:00,  1.16it/s]

tokenizing: 10it [00:00, 217.55it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.79it/s]

tokenizing: 10it [00:00, 439.77it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.68it/s]

tokenizing: 10it [00:00, 370.81it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.85it/s]

tokenizing: 10it [00:00, 321.29it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.64it/s]

tokenizing: 10it [00:00, 291.69it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.81it/s]

tokenizing: 10

100%|███████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.62it/s]

tokenizing: 10it [00:00, 463.63it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.80it/s]

tokenizing: 10it [00:00, 235.95it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.71it/s]

tokenizing: 10it [00:00, 403.41it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.62it/s]

tokenizing: 10it [00:00, 147.08it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.56it/s]

tokenizing: 10it [00:00, 338.34it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.84it/s]

tokenizing: 10it [00:00, 238.89it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:07<00:00,  1.32it/s]

tokenizing: 10it [00:00, 249.03it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:08<00:00,  1.17it/s]

tokenizing: 10

100%|███████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.43it/s]

tokenizing: 10it [00:00, 296.09it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:07<00:00,  1.35it/s]

tokenizing: 10it [00:00, 504.82it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.45it/s]

tokenizing: 10it [00:00, 293.45it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:07<00:00,  1.31it/s]

tokenizing: 10it [00:00, 439.16it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.81it/s]

tokenizing: 10it [00:00, 262.95it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.73it/s]

tokenizing: 10it [00:00, 459.05it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.83it/s]

tokenizing: 10it [00:00, 240.91it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.73it/s]

tokenizing: 10

...

100%|███████████████████████████████████████████████████| 10/10 [00:07<00:00,  1.38it/s]

tokenizing: 10it [00:00, 370.47it/s]

 70%|███████████████████████████████████▋               | 7/10 [15:22<06:37, 132.49s/it]
tokenizing: 10it [00:00, 449.42it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.51it/s]

tokenizing: 10it [00:00, 263.21it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.67it/s]

tokenizing: 10it [00:00, 380.20it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.46it/s]

tokenizing: 10it [00:00, 270.09it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.64it/s]

tokenizing: 10it [00:00, 356.59it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.50it/s]

tokenizing: 10it [00:00, 312.31it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.68it/s]

tokenizing: 10i

100%|███████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.93it/s]

tokenizing: 10it [00:00, 328.16it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.05it/s]

tokenizing: 10it [00:00, 337.37it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.96it/s]

tokenizing: 10it [00:00, 287.59it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.86it/s]

tokenizing: 10it [00:00, 388.15it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.88it/s]

tokenizing: 10it [00:00, 336.72it/s]

 80%|████████████████████████████████████████▊          | 8/10 [17:15<04:12, 126.42s/it]
tokenizing: 10it [00:00, 300.18it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.89it/s]

tokenizing: 10it [00:00, 214.90it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.82it/s]

tokenizing: 10i

100%|███████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.01it/s]

tokenizing: 10it [00:00, 232.75it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.93it/s]

tokenizing: 10it [00:00, 264.12it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.03it/s]

tokenizing: 10it [00:00, 239.34it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.90it/s]

tokenizing: 10it [00:00, 275.28it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.99it/s]

tokenizing: 10it [00:00, 257.53it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.77it/s]

tokenizing: 10it [00:00, 267.47it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.89it/s]

tokenizing: 10it [00:00, 204.10it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.86it/s]

tokenizing: 10

100%|███████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.01it/s]

tokenizing: 10it [00:00, 445.95it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.87it/s]

tokenizing: 10it [00:00, 510.33it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.86it/s]

tokenizing: 10it [00:00, 458.88it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.69it/s]

tokenizing: 10it [00:00, 458.95it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.89it/s]

tokenizing: 10it [00:00, 444.23it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.68it/s]

tokenizing: 10it [00:00, 482.39it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.98it/s]

tokenizing: 10it [00:00, 448.76it/s]

100%|███████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.94it/s]

tokenizing: 10

1

In [78]:
best_model_index

1

In [82]:
best_model

In [83]:
f1_scores

[0.0,
 0.6666666666666666,
 0.6666666666666666,
 0.6666666666666666,
 0.5714285714285714,
 0.6666666666666666,
 0.6666666666666666,
 0.6666666666666666,
 0.0,
 0.0]

In [84]:
df.shape

(159571, 3)